Personalized implementation of the  Koshelev's proposed indifferentiable hashing to elliptic curves (Demonstration for the BLS12-381 curve (case (q % 27) % 9 =10))

-   The proposed implementation is constant-time and about 25% faster thant the original one.
-   Some precomputed constant are necessary (3 Lagrange coefficients !)
-   Functions phi, crtRatio and hprime are merged into one code
-   Benchmark is performed using generated data from the orginal implementation (included in "KoshilevTests.py")



In [57]:
# Personalized implementation :

import KoshilevTests as ts
import gmpy2 as gmp

p  = gmp.mpz(0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab)
b  = 4
w  = gmp.mpz(0x5f19672fdf76ce51ba69c6076a0f77eaddb3a93be6f89688de17d813620a00022e01fffffffefffe)               # Cube root of the Unity
sb = -2                                                                                                        # Square root of b modulo p
m = gmp.mpz(0x1ed1dc57f84bbbf93c928de17f2a481bb970f135468ac0e2d91d6b69703a074740cbda1169ded097612e38e38e387e6) # m =2 * (p - (p % 27)) // 27
c = gmp.mpz(0x443913130e994ba6d702ebe18e46fbe348483b2598700475b98f722777c7c3c7799e9d9c4552a7e40ef87071b647b54) # Cube root of w (9th root of the unity)    
ic = [0x1029dc23a350e5496440404d2fc3f507cee4f01cb7d80ba7871aec5ce6d8efd3b308b842314897fb5f8c07a4b6685e4c,
      0x83ae25363aebd1786f1244fd28226bceacc23beeb292a644fa711b2ecf44a04b7343b1d4437477b3629a94b7d87c1f6]       # [(c**8) % p , (c**7) % p]
LagC = [0xb2cbf441a381c463d9b3ab1600944e4927b477d7327ff5948f01b0cf2bcd7800fdf9d666f0f6a79a46772ea0e65a8b4,
        0xa50693d4e752d69ec5112cdc4e3b054ba96ed9b37a9f1a25deba80d29a603675263d79be3a2d848ddd0338c69b3b123,
        0x11959df2899cba1d7848a41fb8b006c00be0ef84fa7973fd313658fb8a21d9e77cfa6a3eba1c884cd20a928f79c49ea]     # [((1+c+c**2)*gmp.invert(3,p)) % p, 
                                                                                                               # ((1+w*c**2+(w2*c))*(gmp.invert(3,p)**2)) % p, 
                                                                                                               # ((1+(w2*c**2)+(w*c))*(gmp.invert(3,p)**3)) % p]
def hashToCurve(t1,t2):
    v1 = t1 - (w * t1) % p     
    v2 = t1 - (v1 - (t1 << 1)) % p
    v1 = (abs(v1 + 1) - abs(v1) + 1) >> 1
    v2 = (abs(v2 + 1) - abs(v2) + 1) >> 1    
    t0 = w * (v1 + v2 - 3 * v1 * v2) - (v1 + v2) + 1
    v2 = (c * (t2**3)) % p  
    v1 = (c * (t1**3 + v2 ))  % p    
    v2 = (v1 - (c * v2 << 1)) % p  
    den = (v2**2 - (v1 << 1) + 1) % p    
    den8  = (den ** 8) % p
    den16 = (den8 ** 2) % p
    den25 = (den16 * den8 * den) % p
    u  =  b * (((v1 - 1) << 3) * (den + ((v1 - 1) << 1)))  % p
    new_theta = (den16 * pow((u * den25) % p, m ,p)) % p
    num0 =  ( den + ((v1 - 1) << 2) ) % p
    num2 =  ( den + ((v1 - v2) << 1) * (v2 + 1)) % p    
    num1 =  ( den - ((v1 + v2) << 1) * (v2 - 1)) % p
    x  = (((u * den) ** 2) * (new_theta ** 3))  % p	
    x3 = (x ** 3) % p
    v1 = x3 * (x3 + 1) + 1
    v2 = (x3 - 1) * (w * x3 - 1)	
    v3 = 3 - v2 - v1
    t   = (v1 * t0 + v2 * t1  + v3 * t2 ) % p
    num = (v1 * num0 + v2 * num1 + v3 * num2) % p
    v1 = (x * (v1  + v2 * ic[0] + v3 * ic[1] )) % p
    v1 = (LagC[2] * (v1**2) + LagC[1] * v1 + LagC[0]) % p            
    Z = 3  * v1 * den % p
    Y = sb * v1 * num % p  
    X = u * new_theta * t * den % p
    invZ = gmp.invert(Z,p)	
    return X * invZ % p,Y * invZ % p


def benchmark():
    AllPass = True 
    for i in range(100):AllPass = AllPass & (hashToCurve( ts.t1_t2[i][0],ts.t1_t2[i][1]) ==(ts.x_y[i][0],ts.x_y[i][1]))
    return AllPass

print("- Testing Hash to Group procedure : ","\x1b[32mPassed.\x1b[0m" if benchmark() else "\t\t\x1b[91mFailed.\x1b[0m")
print("- Timing for 100 Hash to Group procedure : ")
%timeit benchmark()


- Testing Hash to Group procedure :  Passed.
- Timing for 100 Hash to Group procedure : 
5.84 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
u = -0xd201000000010000
l = u**4 - u**2 + 1
q = ((u - 1)**2 * l) // 3 + u	# q mod 27 = 10
b = 4
r = q % 27   # r=10 for BLS12-381
w = gmp.mpz(0x5f19672fdf76ce51ba69c6076a0f77eaddb3a93be6f89688de17d813620a00022e01fffffffefffe)  
w2 = w**2 % q
sb = -2

if r % 9 == 1:
	m = (q - r) // 27
	z = gmp.mpz(0x443913130e994ba6d702ebe18e46fbe348483b2598700475b98f722777c7c3c7799e9d9c4552a7e40ef87071b647b54)   # z (i.e., zeta from [1, Section 3]) is a primitive 9th root of unity
	z2 = z**2 % q
	c = z
else:
	r = r % 9
	m = (q - r) // 9
	c = w

def phi(t1,t2):
	s1 = (t1**3) % q
	s2 = (t2**3) % q
	s1s1 = (s1**2) % q
	s2s2 = (s2**2) % q
	s1s2 = (s1*s2) % q
	c2 = (c**2) % q
	c3 = (c*c2) % q
	c4 = (c2**2) % q
	a20 = (c2*s1s1) % q
	a11 = (2*c3*s1s2) % q
	a10 = (2*c*s1) % q
	a02 = (c4*s2s2) % q
	a01 = (2*c2*s2) % q
	num0 = (sb*(a20 - a11 + a10 + a02 + a01 - 3)) % q
	num1 = (sb*(-3*a20 + a11 + a10 + a02 - a01 + 1)) % q
	num2 = (sb*(a20 + a11 - a10 - 3*a02 + a01 + 1)) % q  
	den = (a20 - a11 - a10 + a02 - a01 + 1) % q
	return num0,num1,num2,den

def crtRatio(u,v):
# r == 10:
		u2 = u**2 % q
		v2 = v**2 % q
		v4 = v2**2 % q
		v8 = v4**2 % q
		v9 = v*v8 % q
		v16 = v8**2 % q
		v25 = (v9*v16) % q
		return (u*v8*pow((u2*v25),m,q)) % q

def hPrime(num0,num1,num2,den, t1,t2):
	v = (den**2) % q
	u = ((num0**2) - b*v) % q
	th = crtRatio(u,v)   # theta from [1, Section 3]
	v = ((th**3)*v) % q
	L = [t1, (w*t1) % q, (w2*t1) % q]
	L.sort()
	n = L.index(t1)
	if r % 9 == 1:
		u3 = (u**3 % q)
		v3 = (v**3 % q)
		if v3 == u3:
			X = (w**n*th) % q
			if v == u: 
				Y = 1
				Z = 1
			if v == (w*u) % q:
				Y = z
				Z = z
			if v == (w2*u) % q:
				Y = z2
				Z = z2
			Y = (Y*num0) % q
		if v3 == (w*u3) % q:
			X = (th*t1) % q
			zu = (z*u) % q
			if v == zu:
				Y = 1
				Z = 1
			if v == (w*zu) % q:
				Y = z
				Z = z
			if v == (w2*zu) % q:
				Y = z2
				Z = z2
			Y = (Y*num1) % q
		if v3 == (w2*u3) % q:
			X = (th*t2) % q
			z2u = (z2*u) % q
			if v == z2u:
				Y = 1
				Z = 1
			if v == (w*z2u) % q:
				Y = z
				Z = z
			if v == (w2*z2u) % q:
				Y = z2
				Z = z2
			Y = (Y*num2) % q
		# elif is not used to respect constant-time execution in future low-level implementations
		Z = (Z*den) % q
	else:
		if v == u:
			X = ((w**n)*th) % q
			Y = num0
		if v == (w*u) % q:
			X = (th*t1) % q
			Y = num1
		if v == (w2*u) % q:
			X = (th*t2) % q
			Y = num2
		Z = den
	X = (X*den) % q
	return X,Y,Z

def h(t1,t2):
	num0,num1,num2,den = phi(t1,t2)
	X,Y,Z = hPrime(num0,num1,num2,den, t1,t2)
	invZ = gmp.invert(Z,p)	
	return (X * invZ) % q ,(Y * invZ) % q

def benchmark():
    AllPass = True 
    for i in range(100):AllPass = AllPass & (h( ts.t1_t2[i][0],ts.t1_t2[i][1]) ==(ts.x_y[i][0],ts.x_y[i][1]))
    return AllPass

print("- Testing Hash to Group procedure : ","\x1b[32mPassed.\x1b[0m" if benchmark() else "\t\t\x1b[91mFailed.\x1b[0m")
print("- Timing for 100 Hash to Group procedure : ")
%timeit benchmark()

- Testing Hash to Group procedure :  Passed.
- Timing for 100 Hash to Group procedure : 
6.68 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
